# About
In this notebook we'll be building an Alpaca-based dataset containing every comment scraped from r/Hasan_Piker, specifically prepared for finetuning Llama3.1.

WIP

## Setup Environment

In [86]:
# import libs
import json
import pandas as pd
from loguru import logger

## Config
Change these to build different datasets

In [87]:
SUBREDDITS = ['Hasan_Piker']
CLEANED_DATA = '../../json/output/cleaned_data_DEVELOPMENT.json'

In [88]:
with open(CLEANED_DATA, 'r') as f:
  data = json.load(f)

df = pd.DataFrame(data)

# logging
prefilter_len = len(df)
df.head(1)
logger.info(f"Data loaded: {df.shape}")

2024-11-24 12:58:57.029 | INFO     | __main__:<module>:9 - Data loaded: (46, 25)


In [89]:
# only keep the rows where the subreddit is in SUBREDDITS
df = df[df['subreddit'].isin(SUBREDDITS)]

# logging
posts_removed = prefilter_len - len(df)
df.head(1)
logger.info(f"Data filtered: {df.shape}")
logger.info(f"{posts_removed} post{'s' if posts_removed > 1 else ''} removed")

2024-11-24 12:58:57.037 | INFO     | __main__:<module>:7 - Data filtered: (1, 25)
2024-11-24 12:58:57.037 | INFO     | __main__:<module>:8 - 45 posts removed
